# ETL Project - Shrimp - AMC

Team Members: Aaron Karpie, Ken Yeh, Tiffany Cheng

Please find our Audit ETL documentation here: https://drive.google.com/file/d/102Ozk3nAVs0bXdrnmu-X_IGfweC60j04/view?usp=sharing

##### Extract

In [1]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
import numpy as np

In [2]:
# Read AMC data and Shrimp data
amc_csv_file = "AMC.csv"
amc_data_df = pd.read_csv(amc_csv_file)

shrimp_csv_file="shrimp-prices.csv"
shrimp_data_df=pd.read_csv(shrimp_csv_file)

In [3]:
# Initialize Error Count Variables
shrimp_errors = 0
amc_errors=0
merge_errors=0
positive_errors=0
negative_errors=0

In [4]:
# Return dataframes to check
amc_data_df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-01-01,20.620001,22.000000,19.750000,21.370001,15.369904,7641000
1,2014-02-01,21.500000,22.940001,20.299999,22.889999,16.463131,4650100
2,2014-03-01,22.969999,26.680000,22.299999,24.250000,17.441284,14299500
3,2014-04-01,24.200001,25.139999,21.230000,23.139999,16.642939,6583300
4,2014-05-01,23.090000,24.580000,20.990000,22.670000,16.304897,4521300
...,...,...,...,...,...,...,...
83,2020-12-01,4.430000,4.430000,2.080000,2.120000,2.120000,614042200
84,2021-01-01,2.200000,20.360001,1.910000,13.260000,13.260000,4619495600
85,2021-02-01,17.000000,17.250000,5.260000,8.010000,8.010000,3629496600
86,2021-03-01,8.860000,9.450000,7.500000,8.050000,8.050000,412526500


In [5]:
amc_data_df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [6]:
shrimp_data_df

,Month,Price,Change
0,1/1/1991,12.24,0.00
1,2/1/1991,12.90,5.39
2,3/1/1991,13.23,2.56
3,4/1/1991,13.23,0.00
4,5/1/1991,12.68,-3.84
...,...,...,...
356,9/1/2020,11.86,-4.35
357,10/1/2020,11.35,-3.70
358,11/1/2020,11.53,1.59
359,12/1/2020,11.68,1.30


In [27]:
shrimp_data_df.dtypes

Date              datetime64[ns]
Price                    float64
Percent Change           float64
dtype: object

##### Transform

In [7]:
# Shrimp Data Clean up 

try:
    # Renaming Month column header to Date for clarity and consistency with AMC_df
    shrimp_data_df.rename(columns={'Month':'Date'}, inplace=True)

    # Convert Date to datetime object
    shrimp_data_df["Date"]=pd.to_datetime(shrimp_data_df["Date"], errors='raise', dayfirst=False, yearfirst=False, utc=None, format=None, exact=True, unit=None, infer_datetime_format=False, origin='unix', cache=True)

    # Renaming Change header to Percent Change for clarity and dividing by 100 for preciseness and consistency
    shrimp_data_df.rename(columns={'Change':'Percent Change'}, inplace=True)
    shrimp_data_df["Percent Change"]=shrimp_data_df["Percent Change"]/100
except:
    shrimp_error+=1
    
shrimp_data_df.head()

,Date,Price,Percent Change
0,1991-01-01,12.24,0.0000
1,1991-02-01,12.90,0.0539
2,1991-03-01,13.23,0.0256
3,1991-04-01,13.23,0.0000
4,1991-05-01,12.68,-0.0384


In [8]:
#AMC data cleanup
try:
    # Create AMC stock price percent change columns for analysis and consistency using pct_change() function
    amc_data_df["Open_Percent_Change"]=amc_data_df['Open'].pct_change()
    amc_data_df["High_Percent_Change"]=amc_data_df['High'].pct_change()
    amc_data_df["Low_Percent_Change"]=amc_data_df['Low'].pct_change()
    amc_data_df["Close_Percent_Change"]=amc_data_df['Close'].pct_change()
    
    # Converting Date to datetime type for consistency to Shrimp data
    amc_data_df["Date"]=pd.to_datetime(amc_data_df['Date'])
except:
    amc_error+=1

amc_data_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Open_Percent_Change,High_Percent_Change,Low_Percent_Change,Close_Percent_Change
0,2014-01-01,20.620001,22.000000,19.750000,21.370001,15.369904,7641000,NaN,NaN,NaN,NaN
1,2014-02-01,21.500000,22.940001,20.299999,22.889999,16.463131,4650100,0.042677,0.042727,0.027848,0.071128
2,2014-03-01,22.969999,26.680000,22.299999,24.250000,17.441284,14299500,0.068372,0.163034,0.098522,0.059415
3,2014-04-01,24.200001,25.139999,21.230000,23.139999,16.642939,6583300,0.053548,-0.057721,-0.047982,-0.045773
4,2014-05-01,23.090000,24.580000,20.990000,22.670000,16.304897,4521300,-0.045868,-0.022275,-0.011305,-0.020311


In [9]:
# Merging the AMC and Shrimp Data - dropping rows with dates that were not shared to prevent missing values
# Cleaning column headers to be more concise
try:
    merged_amc_shrimp = pd.merge(shrimp_data_df, amc_data_df, on="Date", how="inner")
    merged_amc_shrimp.rename(columns={'Price':'Shrimp_Price_Per_Pound'}, inplace=True)
    merged_amc_shrimp.rename(columns={'Percent Change':'Shrimp Price Percent Change'}, inplace=True)


    #Removing AMC stock % change columns to create clean merged data that allows for easy calculation
    col=merged_amc_shrimp.columns
    merged_amc_shrimp=merged_amc_shrimp[['Date', 'Shrimp_Price_Per_Pound', 'Shrimp Price Percent Change',
           'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

except:
    merged_error +=1
    
merged_amc_shrimp.head()

,Date,Shrimp_Price_Per_Pound,Shrimp Price Percent Change,Open,High,Low,Close,Adj Close,Volume
0,2014-01-01,16.93,0.0036,20.620001,22.000000,19.750000,21.370001,15.369904,7641000
1,2014-02-01,16.98,0.0030,21.500000,22.940001,20.299999,22.889999,16.463131,4650100
2,2014-03-01,17.09,0.0065,22.969999,26.680000,22.299999,24.250000,17.441284,14299500
3,2014-04-01,17.66,0.0334,24.200001,25.139999,21.230000,23.139999,16.642939,6583300
4,2014-05-01,17.97,0.0176,23.090000,24.580000,20.990000,22.670000,16.304897,4521300


In [10]:
# Generating the AMC_Positive_df and AMC_Negative_df by filtering AMC Open Prices Percent Changes
try:
    amc_positive_df=amc_data_df.loc[amc_data_df["Open_Percent_Change"]>0,["Date", "Open", "Open_Percent_Change"]]
    amc_negative_df=amc_data_df.loc[amc_data_df["Open_Percent_Change"]<0,["Date", "Open", "Open_Percent_Change"]]
except:
    positive_errors +=1

    amc_positive_df.head()

In [11]:
try:
    amc_negative_df=amc_data_df.loc[amc_data_df["Open_Percent_Change"]<0,["Date", "Open", "Open_Percent_Change"]]
except:
    negative_errors +=1

amc_negative_df.head()

,Date,Open,Open_Percent_Change
4,2014-05-01,23.09,-0.045868
5,2014-06-01,22.33,-0.032915
7,2014-08-01,22.68,-0.090618
9,2014-10-01,23.00,-0.027484
12,2015-01-01,26.26,-0.001141


### Create database connection and load dataframes into database

In [12]:
# Connecting to postgres to create amc_shrimp database connection and load tables
rds_connection_string = "postgres:postgres@localhost:5432/amc_shrimp_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [13]:
engine.table_names()

['amc_positive_open',
 'amc_negative_open',
 'amc_data',
 'shrimp_data',
 'merged_data']

In [14]:
amc_positive_df.columns

Index(['Date', 'Open', 'Open_Percent_Change'], dtype='object')

In [15]:
amc_positive_df.to_sql(name='amc_positive_open', con=engine, if_exists='replace', index=False)


In [16]:
amc_negative_df.to_sql(name='amc_negative_open', con=engine, if_exists='replace')


In [17]:
amc_data_df.to_sql(name='amc_data', con=engine, if_exists='replace')



In [18]:
shrimp_data_df.to_sql(name='shrimp_data', con=engine, if_exists='replace')


In [19]:
merged_amc_shrimp.to_sql(name='merged_data', con=engine, if_exists='replace')

In [20]:
# Confirming that database and tables were imported correctly
amc_data_df.dtypes

Date                    datetime64[ns]
Open                           float64
High                           float64
Low                            float64
Close                          float64
Adj Close                      float64
Volume                           int64
Open_Percent_Change            float64
High_Percent_Change            float64
Low_Percent_Change             float64
Close_Percent_Change           float64
dtype: object

### Confirm Data has been added 

In [21]:
pd.read_sql_query('select * from amc_positive_open', con=engine).head()

,Date,Open,Open_Percent_Change
0,2014-02-01,21.500000,0.042677
1,2014-03-01,22.969999,0.068372
2,2014-04-01,24.200001,0.053548
3,2014-07-01,24.940001,0.116883
4,2014-09-01,23.650000,0.042769


In [22]:
pd.read_sql_query('select * from amc_negative_open', con=engine).head()

,index,Date,Open,Open_Percent_Change
0,4,2014-05-01,23.09,-0.045868
1,5,2014-06-01,22.33,-0.032915
2,7,2014-08-01,22.68,-0.090618
3,9,2014-10-01,23.00,-0.027484
4,12,2015-01-01,26.26,-0.001141


In [23]:
pd.read_sql_query('select * from amc_data', con=engine).head()

,index,Date,Open,High,Low,Close,Adj Close,Volume,Open_Percent_Change,High_Percent_Change,Low_Percent_Change,Close_Percent_Change
0,0,2014-01-01,20.620001,22.000000,19.750000,21.370001,15.369904,7641000,NaN,NaN,NaN,NaN
1,1,2014-02-01,21.500000,22.940001,20.299999,22.889999,16.463131,4650100,0.042677,0.042727,0.027848,0.071128
2,2,2014-03-01,22.969999,26.680000,22.299999,24.250000,17.441284,14299500,0.068372,0.163034,0.098522,0.059415
3,3,2014-04-01,24.200001,25.139999,21.230000,23.139999,16.642939,6583300,0.053548,-0.057721,-0.047982,-0.045773
4,4,2014-05-01,23.090000,24.580000,20.990000,22.670000,16.304897,4521300,-0.045868,-0.022275,-0.011305,-0.020311


In [24]:
pd.read_sql_query('select * from shrimp_data', con=engine).head()

,index,Date,Price,Percent Change
0,0,1991-01-01,12.24,0.0000
1,1,1991-02-01,12.90,0.0539
2,2,1991-03-01,13.23,0.0256
3,3,1991-04-01,13.23,0.0000
4,4,1991-05-01,12.68,-0.0384


In [25]:
pd.read_sql_query('select * from merged_data', con=engine).head()

,index,Date,Shrimp_Price_Per_Pound,Shrimp Price Percent Change,Open,High,Low,Close,Adj Close,Volume
0,0,2014-01-01,16.93,0.0036,20.620001,22.000000,19.750000,21.370001,15.369904,7641000
1,1,2014-02-01,16.98,0.0030,21.500000,22.940001,20.299999,22.889999,16.463131,4650100
2,2,2014-03-01,17.09,0.0065,22.969999,26.680000,22.299999,24.250000,17.441284,14299500
3,3,2014-04-01,17.66,0.0334,24.200001,25.139999,21.230000,23.139999,16.642939,6583300
4,4,2014-05-01,17.97,0.0176,23.090000,24.580000,20.990000,22.670000,16.304897,4521300


In [28]:
# Print errors counters to check:
print(shrimp_errors)
print(amc_errors)
print(merge_errors)
print(positive_errors)
print(negative_errors)

0
0
0
0
0
